In [1]:
import railrl.misc

In [2]:
#http://localhost:8897/?token=867d5e6b544df64bc4f175f9cc76a060e388c8d68b38eef2

In [3]:
%matplotlib inline

In [4]:
4+4

8

# Create a basic RNN

In [2]:
import tensorflow as tf
import numpy as np

In [8]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [9]:
num_steps = 1
num_lst_units = 5
x_dim = 2
y_dim = 2
learning_rate = 1e-3

In [10]:
cell = tf.nn.rnn_cell.LSTMCell(num_lst_units, state_is_tuple=True)

In [11]:
cell_input = tf.placeholder(
    tf.float32,
    [None, x_dim]
)
cell_state = (tf.placeholder(tf.float32, [None, num_lst_units]),
              tf.placeholder(tf.float32, [None, num_lst_units]))
with tf.variable_scope("foo") as my_scope:
    cell_output = cell(cell_input, cell_state)

In [12]:
rnn_inputs_ph = tf.placeholder(
    tf.float32,
    [None, num_steps, x_dim],
    name='input_placeholder',
)

rnn_inputs = tf.unpack(rnn_inputs_ph, axis=1)

my_scope.reuse_variables()
with tf.variable_scope(my_scope):
    rnn_outputs, final_state = tf.nn.rnn(
        cell,
        rnn_inputs,
        initial_state=cell_state,
        dtype=tf.float32,
        scope=my_scope,
    )

# labels_ph = tf.placeholder(
#     tf.float32,
#     [None, num_steps, y_dim],
#     name='labels_placeholder',
# )
# labels = tf.unpack(labels_ph, axis=1)
# 
# with tf.variable_scope('softmax'):
#     W = tf.get_variable('W', [num_lst_units, y_dim])
#     b = tf.get_variable('b', [y_dim],
#                         initializer=tf.constant_initializer(0.0))
# logits = [tf.matmul(rnn_output, W) + b for rnn_output in rnn_outputs]
# predictions = [tf.nn.softmax(logit) for logit in logits]
# 
# total_loss = tf.nn.sigmoid_cross_entropy_with_logits(logits, labels)
# train_step = tf.train.AdamOptimizer(
#     learning_rate).minimize(
#     total_loss)

In [13]:
sess.run(tf.global_variables_initializer())

In [14]:
batch_size = 4

rnn_input_values = np.random.rand(batch_size, num_steps, x_dim)
input_values = rnn_input_values[:, 0, :]
state_values = (np.zeros((batch_size, num_lst_units)),
                np.zeros((batch_size, num_lst_units)))

In [15]:
print(cell_input, input_values)
print(cell_state, state_values)
print(rnn_inputs, rnn_input_values)
print(cell_output)
print(rnn_outputs)

Tensor("Placeholder:0", shape=(?, 2), dtype=float32) [[ 0.67583134  0.30547555]
 [ 0.00806199  0.51846878]
 [ 0.06714005  0.03481196]
 [ 0.98799726  0.9286102 ]]
(<tf.Tensor 'Placeholder_1:0' shape=(?, 5) dtype=float32>, <tf.Tensor 'Placeholder_2:0' shape=(?, 5) dtype=float32>) (array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]]), array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]]))
[<tf.Tensor 'unpack:0' shape=(?, 2) dtype=float32>] [[[ 0.67583134  0.30547555]]

 [[ 0.00806199  0.51846878]]

 [[ 0.06714005  0.03481196]]

 [[ 0.98799726  0.9286102 ]]]
(<tf.Tensor 'foo/LSTMCell/mul_2:0' shape=(?, 5) dtype=float32>, LSTMStateTuple(c=<tf.Tensor 'foo/LSTMCell/add_1:0' shape=(?, 5) dtype=float32>, h=<tf.Tensor 'foo/LSTMCell/mul_2:0' shape=(?, 5) dtype=float32>))
[<tf.Tensor 'foo_1/foo/LSTMCell/mul_2:0' shape=(?, 5) dtype=fl

In [16]:
cell_output_value = sess.run(
    cell_output,
    {
        cell_input: input_values,
        cell_state: state_values,
    }
)
rnn_output_values, final_state = sess.run(
    [rnn_outputs, final_state],
    {
        rnn_inputs_ph: rnn_input_values,
        cell_state: state_values,
    }
)

In [17]:
print(cell_output_value[0])
print(rnn_output_values)
print(cell_output_value[1])
print(final_state)

[[ 0.01364925 -0.03001923  0.01099671  0.10006332 -0.09449655]
 [ 0.01146973 -0.02243884  0.0202812   0.02001062  0.00931768]
 [ 0.00148686 -0.00457959  0.00133186  0.0112967  -0.00755501]
 [ 0.02792394 -0.03449276  0.03225492  0.14399907 -0.15013254]]
[array([[ 0.01364925, -0.03001923,  0.01099671,  0.10006332, -0.09449655],
       [ 0.01146973, -0.02243884,  0.0202812 ,  0.02001062,  0.00931768],
       [ 0.00148686, -0.00457959,  0.00133186,  0.0112967 , -0.00755501],
       [ 0.02792394, -0.03449276,  0.03225492,  0.14399907, -0.15013254]], dtype=float32)]
LSTMStateTuple(c=array([[ 0.0295302 , -0.06906649,  0.02252459,  0.21047315, -0.15428998],
       [ 0.0256122 , -0.05282619,  0.03538176,  0.04042629,  0.01666637],
       [ 0.00299891, -0.00929151,  0.0026665 ,  0.0226783 , -0.01474612],
       [ 0.07031271, -0.10156184,  0.05848939,  0.31648633, -0.21337438]], dtype=float32), h=array([[ 0.01364925, -0.03001923,  0.01099671,  0.10006332, -0.09449655],
       [ 0.01146973, -0.022